In [ ]:
!ls

In [2]:
#!lscpu
!free -m

              total        used        free      shared  buff/cache   available
Mem:          15885        1110       12866         152        1908       14304
Swap:             0           0           0


# ==============
# START-UP CODE

In [ ]:
! rm cygnus_lib.*; rm mylib.*
! wget https://raw.githubusercontent.com/gmazzitelli/cygno/master/cygnus_lib.py
! wget https://raw.githubusercontent.com/gmazzitelli/cygno/master/mylib.py
import cygnus_lib as cy
import matplotlib.pyplot as plt
import numpy as np    
import ROOT
from root_numpy import hist2array
import mylib as my


# ========================================
# CODE TO SIMULATE NOISE FROM A ECDF FILE

In [ ]:
############################################################
#functions used to simulate noise
############################################################

# created a ECDF (x,y) data
def ecdf(data):
  """ Compute ECDF """
  x = np.sort(data)
  unique, counts = np.unique(data, return_counts=True)
  y = np.cumsum(counts)
  y = y / float(y[-1])
  y = np.insert(y, 0, 0)
  unique = np.insert(unique, 0, unique[0]-1)
  return(unique,y)

# generate simulated data using a ECDF curve (x,y)
def clone_hist(x, y, Nclone):
    s = np.random.uniform(0,1,Nclone)
    idx = np.searchsorted(y, s, side='left')
    return x[idx]

In [ ]:
############################################################
#load ecdf map and generate data
############################################################
import numpy as np

run = 2054

ecdf_map_load = []

filename = str(run)+'_ecdf_map_test.npy'
ecdf_map_load = np.load(filename,allow_pickle = True,fix_imports=True)

print("ECDF matrix size --> "+str(np.shape(ecdf_map_load)))

In [ ]:
############################################################
# generate simulated signal from loaded ecdf_map variable 
############################################################
import time

t0 = time.time()

Nclone = 500
Nx = np.size(ecdf_map_load, 0)
Ny = np.size(ecdf_map_load, 1)
#Nx = 50
#Ny = 50

print("ECDF matrix size --> "+str(Nx)+", "+str(Ny))

img_sim = np.zeros((Nclone, Nx, Ny), dtype=np.uint16)

for i in range(0, Nx):
  if i % 200 == 0:
    print(i)
  for j in range(0, Ny):
    x = ecdf_map_load[i][j][0]
    y = ecdf_map_load[i][j][1]    
    img_sim[:,i,j] = clone_hist(x, y, Nclone)
    
t1 = time.time()-t0  
print(t1)

# ===================================
# THE CODE BELOW IS ONLY FOR TESTING

In [ ]:
################################
#test of the ecdf_map variable
################################
plt.rcParams.update({'font.size': 16})

import matplotlib.pyplot as plt

#choose a pixel
i = 1001
j = 1001

# plot ecdf
plt.figure(figsize=(10, 6))
#plot the ECDF of the chosen pixel
plt.step(ecdf_map_load[i][j][1],ecdf_map_load[i][j][0], linewidth = 3, where = 'post', label = 'real data')
a,b = ecdf(img_sim[:,i,j])
plt.step(b, a, linestyle = ':', linewidth = 3, where = 'post', label = 'sim. data', color = 'orange')
#plt.plot(ecdf_map1[i][j][0],ecdf_map[i][j][1], ':o') 
#plt.xlim(0.9, 1)
plt.grid()
plt.legend()


# plot epdf
plt.figure(figsize=(10, 6))
plt.plot(ecdf_map_load[i][j][0][1:], np.diff(ecdf_map_load[i][j][1]), 'o', label = 'real data')
bins = range(np.min(img_sim[:,i,j])-1,np.max(img_sim[:,i,j])+2)
plt.hist(img_sim[:,i,j], bins=bins, density = 1, align='left', label = 'sim. data', color = 'orange')
plt.grid()
plt.legend()
plt.xticks(bins)

# plot simulated signal
plt.figure(figsize=(10, 6))
plt.plot(img_sim[:,i,j], label = 'sim. data', color = 'orange')
plt.grid()
plt.xlim(0,500)
plt.legend()




In [ ]:
################################
#visualize simulated image
################################

evt = 1

plt.figure(figsize=(10, 10))
plt.imshow(img_sim[evt,:,:], cmap='gray', vmin=95, vmax=104)
plt.colorbar()
print(img_sim[evt,i,j])

evt = 499

plt.figure(figsize=(10, 10))
plt.imshow(img_sim[evt,:,:], cmap='gray', vmin=95, vmax=104)
plt.colorbar()
print(img_sim[evt,i,j])